**Team Name:** DataNerds

**Student Details:**

--------------------------

Name : Reshma Vijay Jawale

Student Id : A0236581B

--------------------------
Name : Aiden Low Yew Woei

Student Id : A0121969W

--------------------------
Name : Raivat Bhupesh Shah 

Student Id : A0184879A

--------------------------

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from IPython.display import display
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Cleaning methods
import library_code.cleaning
import library_code.constants
import library_code.imputation
import library_code.auxiliary

# Scikit learn
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedKFold
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import KFold
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import KNNImputer
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge

from scipy.spatial import Voronoi, voronoi_plot_2d
from matplotlib.patches import Rectangle, Circle
from matplotlib.colors import ListedColormap

from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

# For adaboost
from sklearn.ensemble import AdaBoostClassifier


import os
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

99


In [36]:
df_prices_train = pd.read_csv("data/train.csv")
df_prices_train_demo = pd.read_csv("data/train.csv")
df_prices_test = pd.read_csv("data/test.csv")
df_commercial_centres = pd.read_csv("data/auxiliary-data/auxiliary-data/sg-commerical-centres.csv")
df_train_stations = pd.read_csv("data/auxiliary-data/auxiliary-data/sg-train-stations.csv")
df_shopping_malls = pd.read_csv("data/auxiliary-data/auxiliary-data/sg-shopping-malls.csv")
df_primary_schools = pd.read_csv("data/auxiliary-data/auxiliary-data/sg-primary-schools.csv")
df_secondary_schools = pd.read_csv("data/auxiliary-data/auxiliary-data/sg-secondary-schools.csv")
print("Property Prices train dataset", df_prices_train.shape)
display(df_prices_train.head())
print("Aux: Commercial Centres", df_commercial_centres.shape)
display(df_commercial_centres.head())
print("Aux: Train Stations", df_train_stations.shape)
display(df_train_stations.head())
print("Aux: Shopping Malls", df_shopping_malls.shape)
display(df_shopping_malls.head())
print("Aux: Primary Schools", df_primary_schools.shape)
display(df_primary_schools.head())
print("Aux: Secondary Schools", df_secondary_schools.shape)
display(df_secondary_schools.head())

Property Prices train dataset (26048, 23)


,listing_id,name,street,type,model,market_segment,type_of_area,bedrooms,bathrooms,district,...,lat,lng,tenure,built_year,no_of_units,area_size,eco_category,accessibility,date_listed,price
0,6998418,seascape,57 cove way,condominium,condominium,ocr,strata,3,4.0,4,...,1.239337,103.837487,leasehold/99 years,2011.0,151.0,2336.0,uncategorized,guarded,2021-05-04,5390000.0
1,2046604,la maison,10 moulmein rise,apartment,apartment,ocr,strata,3,3.0,11,...,1.319533,103.847030,freehold,1999.0,24.0,1259.0,uncategorized,guarded,2021-05-07,2310000.0
2,7563513,viva,2 suffolk walk,condominium,condominium,ocr,strata,4,3.0,11,...,1.315656,103.844445,freehold,2012.0,235.0,1959.0,uncategorized,guarded,2021-06-30,5279500.0
3,3071445,urban treasures,205 jalan eunos,condominium,condominium,ocr,strata,3,2.0,14,...,1.329367,103.905791,freehold,NaN,237.0,883.0,uncategorized,guarded,2022-01-02,1843600.0
4,9667539,infini at east coast,east coast road,apartment,apartment,ocr,strata,3,3.0,15,...,1.309176,103.911352,freehold,NaN,36.0,1066.0,uncategorized,guarded,2021-12-24,2262700.0


Aux: Commercial Centres (38, 4)


,name,type,lat,lng
0,Central Business District,CR,1.286768,103.854529
1,Jurong Lake District,CR,1.334085,103.734513
2,Seletar Regional Centre,CR,1.409435,103.866820
3,Tampines Regional Centre,CR,1.353180,103.945188
4,Woodlands Regional Centre,CR,1.436748,103.786485


Aux: Train Stations (166, 6)


...

,name,lat,lng,wikipedia_link
0,100 AM,1.275012,103.841372,NaN
1,313@Somerset,1.301013,103.838540,NaN
2,Aperia,1.309714,103.863837,NaN
3,Balestier Hill Shopping Centre,1.325442,103.842521,NaN
4,Bugis Cube,1.298276,103.855555,NaN


Aux: Primary Schools (191, 3)


,name,lat,lng
0,Admiralty Primary School,1.442941,103.800345
1,Ahmad Ibrahim Primary School,1.433849,103.832710
2,Ai Tong School,1.360713,103.833018
3,Alexandra Primary School,1.291284,103.824066
4,Anchor Green Primary School,1.390380,103.887354


Aux: Secondary Schools (150, 3)


,name,lat,lng
0,Admiralty Secondary School,1.445912,103.802908
1,Ahmad Ibrahim Secondary School,1.436095,103.830055
2,Anderson Secondary School,1.375417,103.851723
3,Anglican High School,1.330218,103.941759
4,Anglo-Chinese School (Barker Road),1.319709,103.835661


In [24]:
df_prices_test.head()

,bedrooms,bathrooms,district,lat,lng,no_of_units,area_size,tenure_60,tenure_99,tenure_freehold,...,planning_area_sembawang,planning_area_sengkang,planning_area_serangoon,planning_area_singapore river,planning_area_southern islands,planning_area_tampines,planning_area_tanglin,planning_area_toa payoh,planning_area_woodlands,planning_area_yishun
0,2.0,2.0,10,1.313566,103.803218,638.0,710.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,3.0,2.0,15,1.298437,103.884408,130.0,1055.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,NaN,1.0,15,1.317851,103.908905,157.0,463.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2.0,1.0,17,1.358005,103.965725,428.0,645.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4.0,6.0,1,1.277474,103.849419,NaN,6200.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Data Cleaning

The data cleaning steps here are based off our EDA notebook.

In [37]:
# First, drop all fields in TO_IGNORE as these columns are either redundant
# e.g. we drop model because we have type
# or they only have one unique value, e.g. 'market_segment' and 'type_of_area'.
df_prices_train = df_prices_train.drop(columns=constants.TO_IGNORE)
df_prices_test = df_prices_test.drop(columns=constants.TO_IGNORE)

In [38]:
# Then, we clean up the tenure fields to 3 categories: 60 years, 99 years, and freehold/999
df_prices_train['tenure'] = df_prices_train['tenure'].apply(cleaning.tenure_to_binary)
df_prices_test['tenure'] = df_prices_test['tenure'].apply(cleaning.tenure_to_binary)
# Perform one hot encoding
df_prices_train = cleaning.categorical_to_onehot(df_prices_train, ['tenure', 'type', 'region', 'planning_area'])
df_prices_test = cleaning.categorical_to_onehot(df_prices_test, ['tenure', 'type', 'region', 'planning_area'])
# Remove 'planning_area_seletar' column in test as that's not seen in train.
df_prices_test.drop(columns=['planning_area_seletar'], inplace=True)

## Add Additional Data

In [26]:
df_consumer_prices = pd.read_csv("data/additional_data/consumer_price.csv")

# Select only the 'all items' data as we need accomodation accounted for in the price index
df_consumer_prices = df_consumer_prices.loc[df_consumer_prices["level_1"] == "All Items"]

# Delete the 'level_1' column now as it is the same across all rows
df_consumer_prices.drop('level_1', axis=1, inplace=True)

# Set 'month' as index because we know it is unique and this will help the look up later when assining
# the consumer price index for each listing
consumer_prices = df_consumer_prices.set_index('month').T.to_dict()

# Assign a consumer price index value for each row in the training and test sets
for index, row in df_prices_train.iterrows():
    curr_listing_date = row['date_listed']
    curr_year_month = curr_listing_date[:7]
    df_prices_train.at[index, 'price_index'] = consumer_prices[curr_year_month]['value']

df_prices_train.head()

,listing_id,name,street,type,model,market_segment,type_of_area,bedrooms,bathrooms,district,...,lng,tenure,built_year,no_of_units,area_size,eco_category,accessibility,date_listed,price,price_index
0,6998418,seascape,57 cove way,condominium,condominium,ocr,strata,3,4.0,4,...,103.837487,leasehold/99 years,2011.0,151.0,2336.0,uncategorized,guarded,2021-05-04,5390000.0,101.883
1,2046604,la maison,10 moulmein rise,apartment,apartment,ocr,strata,3,3.0,11,...,103.847030,freehold,1999.0,24.0,1259.0,uncategorized,guarded,2021-05-07,2310000.0,101.883
2,7563513,viva,2 suffolk walk,condominium,condominium,ocr,strata,4,3.0,11,...,103.844445,freehold,2012.0,235.0,1959.0,uncategorized,guarded,2021-06-30,5279500.0,101.87
3,3071445,urban treasures,205 jalan eunos,condominium,condominium,ocr,strata,3,2.0,14,...,103.905791,freehold,NaN,237.0,883.0,uncategorized,guarded,2022-01-02,1843600.0,104.472
4,9667539,infini at east coast,east coast road,apartment,apartment,ocr,strata,3,3.0,15,...,103.911352,freehold,NaN,36.0,1066.0,uncategorized,guarded,2021-12-24,2262700.0,104.439


In [27]:
for index, row in df_prices_test.iterrows():
    curr_listing_date = row['date_listed']
    curr_year_month = curr_listing_date[:7]
    df_prices_test.at[index, 'price_index'] = consumer_prices[curr_year_month]['value']

df_prices_test.head()

,listing_id,name,street,type,model,market_segment,type_of_area,bedrooms,bathrooms,district,...,lat,lng,tenure,built_year,no_of_units,area_size,eco_category,accessibility,date_listed,price_index
0,1487111,leedon green,leedon heights,condominium,condominium,ocr,strata,2,2.0,10,...,1.313566,103.803218,freehold,NaN,638.0,710.0,uncategorized,guarded,2021-12-12,104.439
1,6794066,the line @ tanjong rhu,tanjong rhu road,condominium,condominium,ocr,strata,3,2.0,15,...,1.298437,103.884408,freehold,2016.0,130.0,1055.0,uncategorized,guarded,2021-10-12,102.95
2,4027017,parc elegance,telok kurau road,apartment,apartment,ocr,strata,NaN,1.0,15,...,1.317851,103.908905,freehold,2013.0,157.0,463.0,uncategorized,guarded,2021-12-23,104.439
3,1119141,the jovell,13 flora drive,condominium,condominium,ocr,strata,2,1.0,17,...,1.358005,103.965725,leasehold/99 years,1930.0,428.0,645.0,uncategorized,guarded,2021-12-31,104.439
4,7374262,shenton way,5 shenton way,condominium,condominium,ocr,strata,4,6.0,1,...,1.277474,103.849419,leasehold/99 years,NaN,NaN,6200.0,uncategorized,guarded,2021-07-14,101.672


# Adding Auxiliary Data

In [6]:
from geopy import distance
from tqdm import tqdm
tqdm.pandas()


def distance_ll(lat1: float, lng1: float, lat2: float, lng2: float) -> float:
    return distance.distance((lat1, lng1), (lat2, lng2))

def dist_to_nearest_item(expr: pd.DataFrame, aux_df: pd.DataFrame) -> str:
    return round(min(aux_df.apply(lambda x: distance_ll(expr['lat'], expr['lng'], x['lat'], x['lng']), axis=1)).km, 1)

# # Add distance to Primary School.
# df_prices_train['dist_primary_school'] = df_prices_train.progress_apply(lambda x: dist_to_nearest_item(x, df_primary_schools), axis=1)
# df_prices_test['dist_primary_school'] = df_prices_test.progress_apply(lambda x: dist_to_nearest_item(x, df_primary_schools), axis=1)


In [7]:
# Add distance to MRT. This might take some time, especially on older machines.  
df_prices_train['dist_MRT'] = df_prices_train.progress_apply(lambda x: dist_to_nearest_item(x, df_train_stations), axis=1)
df_prices_test['dist_MRT'] = df_prices_test.progress_apply(lambda x: dist_to_nearest_item(x, df_train_stations), axis=1)

# Reduce feature dimension size to a binary variable if within 500m of a MRT or not
# Reference: https://www.propertyguru.com.sg/property-guides/hdb-near-mrt-how-much-more-56969#:~:text=On%20average%2C%204%2Droom%20HDB,flats%20without%20a%20nearby%20MRT.
# df_prices_train['dist_MRT'] = df_prices_train['dist_MRT'] < 0.5
# df_prices_test['dist_MRT'] = df_prices_test['dist_MRT'] < 0.5

100%|██████████| 7500/7500 [04:52<00:00, 25.67it/s]


In [12]:
# # Add distance to Shopping Mall.
# df_prices_train['dist_mall'] = df_prices_train.progress_apply(lambda x: dist_to_nearest_item(x, df_shopping_malls), axis=1)
# df_prices_test['dist_mall'] = df_prices_test.progress_apply(lambda x: dist_to_nearest_item(x, df_shopping_malls), axis=1)


100%|███████████████████████████████████████| 7500/7500 [02:09<00:00, 57.73it/s]


In [13]:
# # Add distance to Secondary School.
# df_prices_train['dist_sec_school'] = df_prices_train.progress_apply(lambda x: dist_to_nearest_item(x, df_secondary_schools), axis=1)
# df_prices_test['dist_sec_school'] = df_prices_test.progress_apply(lambda x: dist_to_nearest_item(x, df_secondary_schools), axis=1)


100%|███████████████████████████████████████| 7500/7500 [01:52<00:00, 66.88it/s]


## Cleaning and imputing values for bedroom and bathroom

Some of the values in the bedroom column are math expressions, e.g. '4+1. We're not sure what this means! Is it 5 bedrooms? Is the +1 because it is not a *full room* (e.g. a servant quarter or living room) or is the +1 referring to a bathroom? Due to these different possibilities, we follow an iterative approach where for we take 4+1 as 5, 4.5 (to quantify only half a room) and 4 (to quantify no room). We will go with the representation that gives us the most accuracy

In [39]:
df_prices_train['bedrooms'] = df_prices_train['bedrooms'].apply(cleaning.process_bedroom_sum)
df_prices_test['bedrooms'] = df_prices_test['bedrooms'].apply(cleaning.process_bedroom_sum)
df_prices_train[df_prices_train['bedrooms'].isna()]

,bedrooms,bathrooms,district,lat,lng,no_of_units,area_size,price,tenure_60,tenure_99,...,planning_area_sembawang,planning_area_sengkang,planning_area_serangoon,planning_area_singapore river,planning_area_southern islands,planning_area_tampines,planning_area_tanglin,planning_area_toa payoh,planning_area_woodlands,planning_area_yishun
59,NaN,1.0,15,1.314620,103.932237,116.0,409.0,715000.0,0,0,...,0,0,0,0,0,0,0,0,0,0
111,NaN,NaN,6,1.292988,103.851047,39.0,6609.0,27423000.0,0,1,...,0,0,0,0,0,0,0,0,0,0
203,NaN,1.0,2,1.274644,103.844742,360.0,365.0,990000.0,0,1,...,0,0,0,0,0,0,0,0,0,0
296,NaN,1.0,2,1.274644,103.844742,360.0,365.0,1100000.0,0,1,...,0,0,0,0,0,0,0,0,0,0
443,NaN,NaN,22,1.335557,103.742417,738.0,506.0,1045000.0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25849,NaN,1.0,18,1.376176,103.960488,473.0,571.0,693000.0,0,1,...,0,0,0,0,0,0,0,0,0,0
25866,NaN,2.0,23,1.379727,103.760191,338.0,624.0,825000.0,0,1,...,0,0,0,0,0,0,0,0,0,0
25884,NaN,1.0,1,1.277083,103.849181,510.0,441.0,1099800.0,0,1,...,0,0,0,0,0,0,0,0,0,0
25916,NaN,1.0,7,1.297510,103.856297,522.0,409.0,1098900.0,0,1,...,0,0,0,0,0,0,0,0,0,0


## Dealing with NaN values in Bedrooms, Bathrooms, no_of_units, and area_size column

We have missing values for bedrooms, bathrooms, no_of_units, and area_size column. For handling these, we are using scikit-learn class KNNImputer.

The KNNImputer class provides imputation for filling in missing values using the k-Nearest Neighbors approach. By default, a euclidean distance metric that supports missing values, nan_euclidean_distances, is used to find the nearest neighbors. Each missing feature is imputed using values from n_neighbors nearest neighbors that have a value for the feature. The feature of the neighbors are averaged uniformly or weighted by distance to each neighbor. If a sample has more than one feature missing, then the neighbors for that sample can be different depending on the particular feature being imputed. When the number of available neighbors is less than n_neighbors and there are no defined distances to the training set, the training set average for that feature is used during imputation. If there is at least one neighbor with a defined distance, the weighted or unweighted average of the remaining neighbors will be used during imputation. If a feature is always missing in training, it is removed during transform.

In [11]:
###### SKIP AS SOMEHOW WITHOUT MINMAX SCALING OUR MODEL PERFORMS BETTER??

# We perform KNN imputation with our custom grid search to pick the best parameters k.
# Imputation is done for: 'bedrooms', 'bathrooms', 'no_of_units', 'area_size'
# Reference: https://stackoverflow.com/questions/68072890/filling-in-missing-data-in-pandas-using-knnimputer

# param_grid = ParameterGrid({"n_neighbors": list(range(1, 21, 2)),
#                             "weights": ['uniform', 'distance']})

# ### NORMALIZE with min-max scaling
# train_df_no_price = df_prices_train.drop(columns=['price'])
# min_max_scaler = MinMaxScaler()
# min_max_scaler.fit(train_df_no_price)
# train_df_no_price = pd.DataFrame(min_max_scaler.transform(train_df_no_price), columns=train_df_no_price.columns)
# df_prices_test = pd.DataFrame(min_max_scaler.transform(df_prices_test), columns=df_prices_test.columns)
# df_prices_train.update(train_df_no_price)

### DO IMPUTATION Done in our pipeline below.
# imputer = KNNImputer(n_neighbors=5)
# imputer.fit(train_df_no_price)
# train_df_no_price[:] = imputer.transform(train_df_no_price)
# df_prices_test[:] = imputer.transform(df_prices_test)
# unscale back to original values
# train_df_no_price = pd.DataFrame(min_max_scaler.inverse_transform(train_df_no_price), columns=train_df_no_price.columns)
# df_prices_test = pd.DataFrame(min_max_scaler.inverse_transform(df_prices_test), columns=df_prices_test.columns)

In [12]:

# We want our values to make sense. Manually round and trim values to reduce dimensionality.
# train_df_no_price[['bedrooms', 'bathrooms', 'no_of_units']] = train_df_no_price[['bedrooms', 'bathrooms', 'no_of_units']].clip(lower=1)
# train_df_no_price[['bedrooms', 'bathrooms']] = train_df_no_price[['bedrooms', 'bathrooms']].round()
# df_prices_test[['bedrooms', 'bathrooms', 'no_of_units']] = df_prices_test[['bedrooms', 'bathrooms', 'no_of_units']].clip(lower=1)
# df_prices_test[['bedrooms', 'bathrooms']] = df_prices_test[['bedrooms', 'bathrooms']].round()

# Adaboost

In [13]:
X_train = df_prices_train.loc[:, df_prices_train.columns != 'price']
y_train = df_prices_train['price'].values.ravel()

model = AdaBoostRegressor(base_estimator=DecisionTreeRegressor())
# Set up pipeline with imputer for proper grid search
# Reference: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
pipeline = Pipeline(steps=[('imputer', KNNImputer()), ('model', model)])

# define grid search for hyperparameters
grid = {
    'imputer__n_neighbors': [3, 5, 7, 11, 15],
    'model__base_estimator__max_depth': [i for i in range(1, 21, 2)],
    'model__n_estimators': [50, 100, 150, 250, 500],
    'model__learning_rate': [0.1, 1.0],
}
# Repeats set at 3 to reduce model 
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

mse = make_scorer(mean_squared_error, greater_is_better=False)
grid_search = HalvingGridSearchCV(estimator=pipeline, param_grid=grid, cv=cv, n_jobs=-1, scoring=mse, verbose=1)
# Execute the grid search
grid_results = grid_search.fit(X_train, y_train)

n_iterations: 5
n_required_iterations: 6
n_possible_iterations: 5
min_resources_: 107
max_resources_: 26048
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 500
n_resources: 107
Fitting 10 folds for each of 500 candidates, totalling 5000 fits
----------
iter: 1
n_candidates: 167
n_resources: 321
Fitting 10 folds for each of 167 candidates, totalling 1670 fits
----------
iter: 2
n_candidates: 56
n_resources: 963
Fitting 10 folds for each of 56 candidates, totalling 560 fits
----------
iter: 3
n_candidates: 19
n_resources: 2889
Fitting 10 folds for each of 19 candidates, totalling 190 fits
----------
iter: 4
n_candidates: 7
n_resources: 8667
Fitting 10 folds for each of 7 candidates, totalling 70 fits


/opt/homebrew/Caskroom/miniforge/base/envs/cs5228/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [14]:
# Get RMSE of grid results best score
best_rmse = (-grid_results.best_score_) ** 0.5
print("Best: %f using %s" % (best_rmse, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Best: 1104443.759992 using {'imputer__n_neighbors': 5, 'model__base_estimator__max_depth': 17, 'model__learning_rate': 0.1, 'model__n_estimators': 150}


In [15]:
clf = grid_results
X_test = df_prices_test[:]
y_pred = clf.predict(X_test)

In [16]:
result = pd.DataFrame({"Predicted": y_pred})
result.index.name = "Id"
result.to_csv("submission_adaboost.csv")

# Random Forest

In [17]:
# Get data
X_train = df_prices_train.loc[:, df_prices_train.columns != 'price']
y_train = df_prices_train[['price']].values.ravel()

In [18]:
X_train.shape

(26048, 56)

In [ ]:
model = RandomForestRegressor(n_jobs=-1)
# Set up pipeline with imputer for proper grid search
# Reference: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
pipeline = Pipeline(steps=[('imputer', KNNImputer()), ('model', model)])

# define grid search for hyperparameters
grid = {
    'imputer__n_neighbors': [3, 5, 7, 11, 15],
    'model__max_features': ['auto', 'sqrt'],
    'model__max_depth': [i for i in range(1, 21, 2)],
    'model__n_estimators': [25, 50, 100, 150, 250, 500],
}


cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
# define grid search for hyperparameters
num_features = X_train.shape[1]

mse = make_scorer(mean_squared_error, greater_is_better=False)
grid_search = HalvingGridSearchCV(estimator=pipeline, param_grid=grid, n_jobs=-1, cv=cv, scoring=mse, verbose=1)
# Execute the grid search
grid_results = grid_search.fit(X_train, y_train)

In [20]:
# Get RMSE of grid results best score
best_rmse = (-grid_results.best_score_) ** 0.5
print("Best: %f using %s" % (best_rmse, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Best: 1094381.235749 using {'imputer__n_neighbors': 7, 'model__max_depth': 15, 'model__max_features': 'auto', 'model__n_estimators': 150}


In [21]:
clf = grid_results
X_test = df_prices_test.loc[:, df_prices_test.columns != 'price']
y_pred = clf.predict(X_test)

In [22]:
result = pd.DataFrame({"Predicted": y_pred})
result.index.name = "Id"
result.to_csv("submission_forest.csv")

# Bagging Regressor

In [27]:
# Get data
X_train = df_prices_train.loc[:, df_prices_train.columns != 'price']
y_train = df_prices_train[['price']].values.ravel()
print(X_train.shape)
model = BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_jobs=-1)
# Set up pipeline with imputer for proper grid search
# Reference: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
pipeline = Pipeline(steps=[('imputer', KNNImputer()), ('model', model)])

# define grid search for hyperparameters
grid = {
    'imputer__n_neighbors': [3, 5, 7, 11, 15],
    'model__max_features': [0.5, 0.7, 0.9, 1],
    'model__base_estimator__max_depth': [i for i in range(1, 21, 2)],
    'model__n_estimators': [50, 100, 150, 250, 500],
}


cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
# define grid search for hyperparameters
num_features = X_train.shape[1]

mse = make_scorer(mean_squared_error, greater_is_better=False)
grid_search = HalvingGridSearchCV(estimator=pipeline, param_grid=grid, n_jobs=-1, cv=cv, scoring=mse, verbose=1)
# Execute the grid search
grid_results = grid_search.fit(X_train, y_train)

(26048, 56)
n_iterations: 7
n_required_iterations: 7
n_possible_iterations: 7
min_resources_: 35
max_resources_: 26048
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 1000
n_resources: 35
Fitting 10 folds for each of 1000 candidates, totalling 10000 fits
----------
iter: 1
n_candidates: 334
n_resources: 105
Fitting 10 folds for each of 334 candidates, totalling 3340 fits
----------
iter: 2
n_candidates: 112
n_resources: 315
Fitting 10 folds for each of 112 candidates, totalling 1120 fits
----------
iter: 3
n_candidates: 38
n_resources: 945
Fitting 10 folds for each of 38 candidates, totalling 380 fits
----------
iter: 4
n_candidates: 13
n_resources: 2835
Fitting 10 folds for each of 13 candidates, totalling 130 fits
----------
iter: 5
n_candidates: 5
n_resources: 8505
Fitting 10 folds for each of 5 candidates, totalling 50 fits


/opt/homebrew/Caskroom/miniforge/base/envs/cs5228/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


----------
iter: 6
n_candidates: 2
n_resources: 25515
Fitting 10 folds for each of 2 candidates, totalling 20 fits


In [28]:
# Get RMSE of grid results best score
best_rmse = (-grid_results.best_score_) ** 0.5
print("Best: %f using %s" % (best_rmse, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Best: 800328.243916 using {'imputer__n_neighbors': 3, 'model__base_estimator__max_depth': 15, 'model__max_features': 0.9, 'model__n_estimators': 250}


In [29]:
# Predict
clf = grid_results
X_test = df_prices_test.loc[:, df_prices_test.columns != 'price']
y_pred = clf.predict(X_test)

In [30]:
result = pd.DataFrame({"Predicted": y_pred})
result.index.name = "Id"
result.to_csv("submission_bagging.csv")

# GradientBoostingRegressor

In [20]:
# Get data
X_train = df_prices_train.loc[:, df_prices_train.columns != 'price']
y_train = df_prices_train[['price']].values.ravel()
print(X_train.shape)
model = GradientBoostingRegressor()
# Set up pipeline with imputer for proper grid search
# Reference: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
pipeline = Pipeline(steps=[('imputer', KNNImputer()), ('model', model)])

# define grid search for hyperparameters
grid = {
    'imputer__n_neighbors': [3, 5, 7, 11, 15],
    'model__subsample': [0.2, 0.5, 0.9, 1],
    'model__max_features': ['auto', 'sqrt'],
    'model__max_depth': [i for i in range(1, 21, 2)],
    'model__n_estimators': [50, 100, 150, 250, 500],
}


cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
# define grid search for hyperparameters
num_features = X_train.shape[1]

mse = make_scorer(mean_squared_error, greater_is_better=False)
grid_search = HalvingGridSearchCV(estimator=pipeline, param_grid=grid, n_jobs=-1, cv=cv, scoring=mse, verbose=1)
# Execute the grid search
grid_results = grid_search.fit(X_train, y_train)

(26048, 56)
n_iterations: 7
n_required_iterations: 7
n_possible_iterations: 7
min_resources_: 35
max_resources_: 26048
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 2000
n_resources: 35
Fitting 10 folds for each of 2000 candidates, totalling 20000 fits
----------
iter: 1
n_candidates: 667
n_resources: 105
Fitting 10 folds for each of 667 candidates, totalling 6670 fits
----------
iter: 2
n_candidates: 223
n_resources: 315
Fitting 10 folds for each of 223 candidates, totalling 2230 fits
----------
iter: 3
n_candidates: 75
n_resources: 945
Fitting 10 folds for each of 75 candidates, totalling 750 fits
----------
iter: 4
n_candidates: 25
n_resources: 2835
Fitting 10 folds for each of 25 candidates, totalling 250 fits
----------
iter: 5
n_candidates: 9
n_resources: 8505
Fitting 10 folds for each of 9 candidates, totalling 90 fits
----------
iter: 6
n_candidates: 3
n_resources: 25515
Fitting 10 folds for each of 3 candidates, totalling 30 fits


In [30]:
# Get RMSE of grid results best score
best_rmse = (-grid_results.best_score_) ** 0.5
print("Best: %f using %s" % (best_rmse, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Best: 747393.545766 using {'imputer__n_neighbors': 3, 'model__max_depth': 5, 'model__max_features': 'auto', 'model__n_estimators': 250, 'model__subsample': 0.5}


In [33]:
# Predict
clf = grid_results
X_test = df_prices_test.loc[:, df_prices_test.columns != 'price']
y_pred = clf.predict(X_test)

In [34]:
result = pd.DataFrame({"Predicted": y_pred})
result.index.name = "Id"
result.to_csv("submission_gradient_boosting_no_aux_add_inflation.csv")

# Linear Regression

In [45]:
# Get data
X_train = df_prices_train.loc[:, df_prices_train.columns != 'price']
y_train = df_prices_train[['price']].values.ravel()
print(X_train.shape)
model = LinearRegression()
# Set up pipeline with imputer for proper grid search
# Reference: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
pipeline = Pipeline(steps=[('imputer', KNNImputer()), ('model', model)])

# define grid search for hyperparameters
grid = {
    'imputer__n_neighbors': [3, 5, 7, 11, 15],
    'model__fit_intercept': [False, True],
}


cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
# define grid search for hyperparameters
num_features = X_train.shape[1]

mse = make_scorer(mean_squared_error, greater_is_better=False)
grid_search = HalvingGridSearchCV(estimator=pipeline, param_grid=grid, n_jobs=-1, cv=cv, scoring=mse, verbose=1)
# Execute the grid search
grid_results = grid_search.fit(X_train, y_train)

(26048, 56)
n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 2894
max_resources_: 26048
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 10
n_resources: 2894
Fitting 10 folds for each of 10 candidates, totalling 100 fits
----------
iter: 1
n_candidates: 4
n_resources: 8682
Fitting 10 folds for each of 4 candidates, totalling 40 fits


/opt/homebrew/Caskroom/miniforge/base/envs/cs5228/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


----------
iter: 2
n_candidates: 2
n_resources: 26046
Fitting 10 folds for each of 2 candidates, totalling 20 fits


In [46]:
# Get RMSE of grid results best score
best_rmse = (-grid_results.best_score_) ** 0.5
print("Best: %f using %s" % (best_rmse, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Best: 1909914.525339 using {'imputer__n_neighbors': 11, 'model__fit_intercept': True}


In [47]:
# Predict
clf = grid_results
X_test = df_prices_test.loc[:, df_prices_test.columns != 'price']
y_pred = clf.predict(X_test)

In [48]:
result = pd.DataFrame({"Predicted": y_pred})
result.index.name = "Id"
result.to_csv("submission_linear_regression.csv")

# Bayesian Regression

In [49]:
# Get data
X_train = df_prices_train.loc[:, df_prices_train.columns != 'price']
y_train = df_prices_train[['price']].values.ravel()
print(X_train.shape)
model = BayesianRidge()
# Set up pipeline with imputer for proper grid search
# Reference: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
pipeline = Pipeline(steps=[('imputer', KNNImputer()), ('model', model)])

# define grid search for hyperparameters
grid = {
    'imputer__n_neighbors': [3, 5, 7, 11, 15],
}


cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
# define grid search for hyperparameters
num_features = X_train.shape[1]

mse = make_scorer(mean_squared_error, greater_is_better=False)
grid_search = HalvingGridSearchCV(estimator=pipeline, param_grid=grid, n_jobs=-1, cv=cv, scoring=mse, verbose=1)
# Execute the grid search
grid_results = grid_search.fit(X_train, y_train)

(26048, 56)
n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 8682
max_resources_: 26048
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 5
n_resources: 8682
Fitting 10 folds for each of 5 candidates, totalling 50 fits
----------
iter: 1
n_candidates: 2
n_resources: 26046
Fitting 10 folds for each of 2 candidates, totalling 20 fits


In [58]:
# Get RMSE of grid results best score
best_rmse = (-grid_results.best_score_) ** 0.5
print("Best: %f using %s" % (best_rmse, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Best: 1910489.589151 using {'imputer__n_neighbors': 15}


In [51]:
# Predict
clf = grid_results
X_test = df_prices_test.loc[:, df_prices_test.columns != 'price']
y_pred = clf.predict(X_test)

In [52]:
result = pd.DataFrame({"Predicted": y_pred})
result.index.name = "Id"
result.to_csv("submission_bayesian_regression.csv")

# KNeighbors Regression

In [40]:
# Get data
X_train = df_prices_train.loc[:, df_prices_train.columns != 'price']
y_train = df_prices_train[['price']].values.ravel()
print(X_train.shape)
model = KNeighborsRegressor()
# Set up pipeline with imputer for proper grid search
# Reference: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
pipeline = Pipeline(steps=[('imputer', KNNImputer()), ('model', model)])

# define grid search for hyperparameters
grid = {
    'imputer__n_neighbors': [3, 5, 7, 11, 15],
    'model__n_neighbors': [i for i in range(1, 32, 2)]
}


cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
# define grid search for hyperparameters
num_features = X_train.shape[1]

mse = make_scorer(mean_squared_error, greater_is_better=False)
grid_search = HalvingGridSearchCV(estimator=pipeline, param_grid=grid, n_jobs=-1, cv=cv, scoring=mse, verbose=1)
# Execute the grid search
grid_results = grid_search.fit(X_train, y_train)

(26048, 55)
n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 964
max_resources_: 26048
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 80
n_resources: 964
Fitting 10 folds for each of 80 candidates, totalling 800 fits
----------
iter: 1
n_candidates: 27
n_resources: 2892
Fitting 10 folds for each of 27 candidates, totalling 270 fits
----------
iter: 2
n_candidates: 9
n_resources: 8676
Fitting 10 folds for each of 9 candidates, totalling 90 fits
----------
iter: 3
n_candidates: 3
n_resources: 26028
Fitting 10 folds for each of 3 candidates, totalling 30 fits


exception calling callback for <Future at 0x7f1f584c2790 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/root/data-mining-property-prices/env/lib/python3.9/site-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/root/data-mining-property-prices/env/lib/python3.9/site-packages/joblib/parallel.py", line 359, in __call__
    self.parallel.dispatch_next()
  File "/root/data-mining-property-prices/env/lib/python3.9/site-packages/joblib/parallel.py", line 794, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/root/data-mining-property-prices/env/lib/python3.9/site-packages/joblib/parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "/root/data-mining-property-prices/env/lib/python3.9/site-packages/joblib/parallel.py", line 779, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/root/data-mining-property-prices/env

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}

In [60]:
# Get RMSE of grid results best score
best_rmse = (-grid_results.best_score_) ** 0.5
print("Best: %f using %s" % (best_rmse, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Best: 1194467.214621 using {'imputer__n_neighbors': 5}


In [54]:
# Predict
clf = grid_results
X_test = df_prices_test.loc[:, df_prices_test.columns != 'price']
y_pred = clf.predict(X_test)

In [55]:
result = pd.DataFrame({"Predicted": y_pred})
result.index.name = "Id"
result.to_csv("submission_kneighbors.csv")